In [1]:
!pip install fasttext
!pip install fastparquet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 888.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227138 sha256=795cc2b0b1b573c004ea820ca672cd226b97a5cb8a77fcb70057ef22521af2a1
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.5 MB/s eta 0:00:00


In [46]:
from gensim.models import keyedvectors
from scipy.spatial import distance
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
import pandas as pd
import numpy as np
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader, random_split

In [28]:
tokenizer = AutoTokenizer.from_pretrained("razent/SciFive-large-Pubmed_PMC-MedNLI")
SciFive_model = AutoModelForSeq2SeqLM.from_pretrained("razent/SciFive-large-Pubmed_PMC-MedNLI")
#model.cuda()

sent_1 = "the patient prescribed by antibiotic"
sent_2 = "The patient has an infection"
text =  f"mednli: sentence1: {sent_1} sentence2: {sent_2}"

encoding = tokenizer.encode_plus(text, padding='max_length', max_length=256, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"], encoding["attention_mask"]

outputs = SciFive_model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=8,
    early_stopping=True
)

for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(line)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


entailment


In [14]:
test_data = pd.read_parquet('test-00000-of-00001-47685aa42db61e77.parquet', engine='fastparquet')
train_data = pd.read_parquet('train-00000-of-00001-210cfe9263b99806.parquet', engine='fastparquet')
valid_data = pd.read_parquet('valid-00000-of-00001-cc552de6d1a6fa4b.parquet', engine='fastparquet')
test_data

,id,query,answer,choices,gold
0,MedNLI0,\nTASK: Please classify the relationship betwe...,entailment,"[entailment, contradiction, neutral]",0
1,MedNLI1,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1
2,MedNLI2,\nTASK: Please classify the relationship betwe...,neutral,"[entailment, contradiction, neutral]",2
3,MedNLI3,\nTASK: Please classify the relationship betwe...,entailment,"[entailment, contradiction, neutral]",0
4,MedNLI4,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1
...,...,...,...,...,...
1417,MedNLI1417,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1
1418,MedNLI1418,\nTASK: Please classify the relationship betwe...,neutral,"[entailment, contradiction, neutral]",2
1419,MedNLI1419,\nTASK: Please classify the relationship betwe...,entailment,"[entailment, contradiction, neutral]",0
1420,MedNLI1420,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1


In [9]:
def find_pre_and_hyp(query):
    start_pre = query.find("[PRE]") + len("[PRE]")
    end_pre = query.find("[HYP]")
    start_hyp = query.find("[HYP]") + len("[HYP]")
    end_hyp = query.find("OUTPUT:")
    premise = query[start_pre:end_pre].strip()
    hypothesis = query[start_hyp:end_hyp].strip()

    return premise,hypothesis

In [10]:
def get_answer(premise,hypothesis):
    text =  f"mednli: sentence1: {premise} sentence2: {hypothesis}"
    encoding = tokenizer.encode_plus(text, padding='max_length', max_length=256, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"], encoding["attention_mask"]
    outputs = SciFive_model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=8,
    early_stopping=True)
    for output in outputs:
        line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return line

In [ ]:
count = 0
r = 0
for query,answer in zip(test_data['query'],test_data['answer']):
    count = count + 1
    premise,hypothesis = find_pre_and_hyp(query)
    an = get_answer(premise,hypothesis)
    if an == answer:
       r = r + 1
print(r/count)

In [11]:
text = 'TASK: Please classify the relationship between the given premise and hypothesis into one of the following labels: entailment, contradiction, or neutral. Return only the label. ### INPUT: [PRE] HISTORY: [**Known patient lastname **] is a 33-6/7 weeks male, twin #2, born at 0307 p.m. on [**2749-6-23**] via C-section for preeclampsia to a 34- year-old, G1, para 0, now 2, mother with an [**Name (NI) 2016**] of [**2749-8-5**]. [HYP] the patient is full term OUTPUT: '
premise,hypothesis = find_pre_and_hyp(text)
print(premise)
print(hypothesis)

HISTORY: [**Known patient lastname **] is a 33-6/7 weeks male, twin #2, born at 0307 p.m. on [**2749-6-23**] via C-section for preeclampsia to a 34- year-old, G1, para 0, now 2, mother with an [**Name (NI) 2016**] of [**2749-8-5**].
the patient is full term


In [29]:
tokenizer = AutoTokenizer.from_pretrained("gsarti/biobert-nli")
sen2vec_model = AutoModel.from_pretrained("gsarti/biobert-nli")
#model.cuda()

sent_1 = "the patient hate antibiotic"
sent_2 = "The patient has an infection"
text =  f"mednli: sentence1: {sent_1} sentence2: {sent_2}"

s1 = tokenizer(premise,return_tensors="pt")
s2 = tokenizer(hypothesis,return_tensors="pt")
vec_1 = sen2vec_model(**s1)
vec_2 = sen2vec_model(**s2)

vecc_1 = vec_1[1][0].detach().numpy()
vecc_2 = vec_2[1][0].detach().numpy()

cnc_vec = np.concatenate((vecc_1, vecc_2))
cosine_sim = 1 - distance.cosine(vecc_1, vecc_2)
print('cosine similarity:', cosine_sim)

cosine similarity: 0.3269841969013214


In [30]:
print(len(cnc_vec))
print(len(vecc_1))
print(len(vecc_1))

1536
768
768


In [31]:
class CustomDataset(Dataset):
    def __init__(self, x_list, y_list):
        self.samples = []
        for x,y in zip(x_list,y_list):
            x_tensor = torch.tensor(x,dtype = torch.float32)
            y_tensor = torch.tensor(y,dtype = torch.float32)
            self.samples.append((x_tensor,y_tensor))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

In [50]:
x_list = []
y_list = []
for query,answer in zip(train_data['query'],train_data['answer']):
    if answer == 'entailment':
       y = [1,0,0]
    elif answer == 'neutral':
       y = [0,1,0]
    elif answer == 'contradiction':
       y = [0,0,1]
    else:
        print('should not get here')

    premise,hypothesis = find_pre_and_hyp(query)
    s1 = tokenizer(premise,return_tensors="pt")
    s2 = tokenizer(hypothesis,return_tensors="pt")
    vec_1 = sen2vec_model(**s1)
    vec_2 = sen2vec_model(**s2)
    vecc_1 = vec_1[1][0].detach().numpy()
    vecc_2 = vec_2[1][0].detach().numpy()
    x = np.concatenate((vecc_1, vecc_2))

    x_list.append(x)
    y_list.append(y)

In [43]:
print(len(train_data))

11232


In [52]:
train_dataset = CustomDataset(x_list,y_list)

In [166]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(1536, 500)
        self.bn1 = nn.BatchNorm1d(500)
        self.fc2 = nn.Linear(500, 200)
        self.bn2 = nn.BatchNorm1d(200)
        self.fc3 = nn.Linear(200, 100)
        self.bn3 = nn.BatchNorm1d(100)
        self.fc4 = nn.Linear(100, 50)
        self.bn4 = nn.BatchNorm1d(50)
        self.fc5 = nn.Linear(50, 3)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = self.bn1(x)
        x = torch.tanh(self.fc2(x))
        x = self.bn2(x)
        x = torch.tanh(self.fc3(x))
        x = self.bn3(x)
        x = torch.tanh(self.fc4(x))
        x = self.bn4(x)
        x = nn.functional.softmax(self.fc5(x))
        return x

In [169]:
transform = transforms.Compose([
    transforms.ToTensor()
])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1024, shuffle=True)
# test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)

In [170]:
model = NN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [171]:
model.train()
num_epochs = 2000
for epoch in range(num_epochs):
    for sente, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(sente)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        #if (i+1) % 100 == 0:
            #print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item()}')
    print(loss.item())

print('Training complete.')

<ipython-input-166-5b445ac8e524>:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.softmax(self.fc5(x))


0.9061316251754761
0.8843598961830139
0.8648734092712402
0.845257580280304
0.8790310025215149
0.8454248905181885
0.8176697492599487
0.8359851837158203
0.8081114888191223
0.8076491355895996
0.7798902988433838
0.7592192888259888
0.7473549842834473
0.7384872436523438
0.75857013463974
0.7603501081466675
0.7457093596458435
0.725833535194397
0.7227075695991516
0.7245646119117737
0.7009070515632629
0.7169989943504333
0.7060359716415405
0.6996338367462158
0.6852096915245056
0.674544095993042
0.690658688545227
0.6870543360710144
0.6808660626411438
0.6826561093330383
0.676261305809021
0.67262864112854
0.6796817779541016
0.6699053049087524
0.656233549118042
0.6640235185623169
0.6736311316490173
0.6667000651359558
0.6586175560951233
0.6613309979438782
0.6656649112701416
0.6397616863250732
0.66871178150177
0.6641865372657776
0.6527636647224426
0.6538799405097961
0.6677128672599792
0.6742594838142395
0.6517662405967712
0.6605116724967957
0.6619237661361694
0.668663740158081
0.6874857544898987
0.6468

KeyboardInterrupt: 

In [172]:
count = 0
r = 0
model.eval()
for query,answer in zip(test_data['query'],test_data['answer']):
    count = count + 1
    premise,hypothesis = find_pre_and_hyp(query)

    s1 = tokenizer(premise,return_tensors="pt")
    s2 = tokenizer(hypothesis,return_tensors="pt")
    #print(premise)
    #print(hypothesis)
    vec_1 = sen2vec_model(**s1)
    vec_2 = sen2vec_model(**s2)
    vecc_1 = vec_1[1][0].detach().numpy()
    vecc_2 = vec_2[1][0].detach().numpy()
    x = np.concatenate((vecc_1, vecc_2))
    x_tensor = torch.tensor(x,dtype = torch.float32).unsqueeze(1).expand(-1,2)
    x_tensor = torch.transpose(x_tensor,0,1)
    #print(x_tensor)


    a = np.argmax(model(x_tensor).detach().numpy()[0])
    if a == 0:
       #print('entailment')
       an = 'entailment'
    elif a == 1:
       #print('neutral')
       an = 'neutral'
    elif a == 2:
       #print('contradiction')
       an = 'contradiction'
    else:
       print('should not get  here')

    #print(answer)
    #print('=====================================================================')
    if an == answer:
       r = r + 1
    #print(r)
    #print(count)
print(r/count)

<ipython-input-166-5b445ac8e524>:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.softmax(self.fc5(x))


0.6715893108298172


In [162]:
torch.save(model,'68.84accuracy.pth')